In [1]:
import pandas as pd
import requests
import bs4
import re

# Scrape coding bootcamp rankings from coursereport.com


## Steps:
- 1. Send `GET` request to coursereport.com/best-coding-bootcamps to get the html
- 2. Throw html into a soup
- 3. Extract desired data
    - rank
    - name
    - avg. rating
    - stars
    - no. of reviews
    - locations (3)
    - description
- 4. Throw into DataFrame

### 1. Get html

In [2]:
url = "https://www.coursereport.com/best-coding-bootcamps"

In [3]:
resp = requests.get(url)

In [4]:
type(resp)

requests.models.Response

In [5]:
resp.content[:10]

b'<!DOCTYPE '

### 2. Create a soup

In [6]:
soup = bs4.BeautifulSoup(resp.content, "html.parser")

In [7]:
type(soup)

bs4.BeautifulSoup

In [8]:
soup.body

<body><nav class="navbar navbar-cr navbar-fixed-top" role="navigation"><div class="container-fluid" itemscope="" itemtype="http://schema.org/Organization"><div class="navbar-header"><button aria-expanded="false" class="navbar-toggle collapsed" data-target="#navbar-collapse" data-toggle="collapse" type="button"><span class="sr-only">Toggle navigation</span><span class="icon-bar"></span><span class="icon-bar"></span><span class="icon-bar"></span></button><a class="navbar-brand" href="/" itemprop="url"><svg height="40px" width="113px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><image height="40px" width="113px" x="0px" xlink:href="..." y="0px"/></svg></a></div><div class="collapse navbar-collapse" id="navbar-collapse"><ul class="nav navbar-nav navbar-right"><li><a href="/login">Sign In</a></li></ul><form accept-charset="UTF-8" action="/results" class="navbar-form navbar-right" method="get"><input name="utf8" type="hidden" value="✓"/><div class="form-grou

In [9]:
soup.body.find("div")
# soup.body.div

<div class="container-fluid" itemscope="" itemtype="http://schema.org/Organization"><div class="navbar-header"><button aria-expanded="false" class="navbar-toggle collapsed" data-target="#navbar-collapse" data-toggle="collapse" type="button"><span class="sr-only">Toggle navigation</span><span class="icon-bar"></span><span class="icon-bar"></span><span class="icon-bar"></span></button><a class="navbar-brand" href="/" itemprop="url"><svg height="40px" width="113px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><image height="40px" width="113px" x="0px" xlink:href="..." y="0px"/></svg></a></div><div class="collapse navbar-collapse" id="navbar-collapse"><ul class="nav navbar-nav navbar-right"><li><a href="/login">Sign In</a></li></ul><form accept-charset="UTF-8" action="/results" class="navbar-form navbar-right" method="get"><input name="utf8" type="hidden" value="✓"/><div class="form-group has-feedback"><input data-auto-focus="true" data-autocomplete="/searc

In [10]:
# find div tag with class="main-body"
(soup
 .body
 .find("div", class_="main-body"))

<div class="main-body"><div class="sidebar"><div class="sidebar-apply-module"><a data-ga='{"category":"Click Match Link","action":"Click","label":"https://www.coursereport.com/best-coding-bootcamps"}' href="/get-matched"><span class="top">Not sure what you're <br/> looking for?</span><span class="bottom">We'll match you!</span></a></div></div><div class="longform-body container"><p class="details"></p><span class="author"><span class="icon-user"></span> By: The Course Report Team<h4 class="text-left">Last Update: 10/14/2019</h4></span><div class="row"><div class="col-md-11"><p>As the <a href="/about">leading authority</a> on coding bootcamps, the team at Course Report has been researching, tracking, and sharing the latest on the coding bootcamp industry since 2013. And each year, we use that knowledge and expertise to curate a list of the top 48 in-person coding bootcamps around the world. </p><p>In addition to reading reviews, visiting classrooms, and researching job outcomes, this li

In [11]:
# within div tag with class="main-body", find div tag with 
# class="longform-body container"

(soup
 .body
 .find("div", class_="main-body")
 .find("div", class_="longform-body container"))

<div class="longform-body container"><p class="details"></p><span class="author"><span class="icon-user"></span> By: The Course Report Team<h4 class="text-left">Last Update: 10/14/2019</h4></span><div class="row"><div class="col-md-11"><p>As the <a href="/about">leading authority</a> on coding bootcamps, the team at Course Report has been researching, tracking, and sharing the latest on the coding bootcamp industry since 2013. And each year, we use that knowledge and expertise to curate a list of the top 48 in-person coding bootcamps around the world. </p><p>In addition to reading reviews, visiting classrooms, and researching job outcomes, this list can be a useful tool in creating your shortlist as you <a href="/get-matched">search for the best bootcamp</a> for you.</p><p>To create this list, we considered immersive (full-time, in-person) web and mobile development bootcamps around the world. In past years, Course Report had only published an alphabetized list of select bootcamps; the

In [12]:
school_list_items = (soup
                     .body
                     .find("div", class_="main-body")
                     .find("div", class_="longform-body container")
                     .find("div", class_="row")
                     .find("div", class_="col-md-11")
                     .find("ul", id="schools")
                     .find_all("li"))

Nicholas asked: "Do we have to walk down the tree?"

-- No.

In [13]:
school_list_items = soup.find("ul", id="schools").find_all("li")

# 3. Extract data

**Get rank and name**

In [14]:
sample = school_list_items[0]

In [15]:
sample.div.h3.a

<a href="/schools/flatiron-school">1. Flatiron School</a>

In [16]:
sample.div.h3.a.contents

['1. Flatiron School']

In [17]:
sample.div.h3.a.text

'1. Flatiron School'

**Get avg. rating**

In [18]:
sample.find("p", class_="rating-number").text

'Avg Rating: 4.69 (445 reviews)'

**Get stars**

In [19]:
sample.find("p", class_="ratings").find_all("span")

[<span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-half_star"></span>]

**Get no. of reviews**

In [20]:
sample.find("p", class_="rating-number").text

'Avg Rating: 4.69 (445 reviews)'

**Get locations**

In [21]:
sample.find("span", class_="location").find_all("a")

[<a href="/cities/brooklyn">Brooklyn</a>,
 <a href="/cities/washington-coding-bootcamps">Washington</a>,
 <a href="/cities/houston">Houston</a>]

In [22]:
sample.find("span", class_="location").find_all("a")[0].text

'Brooklyn'

In [23]:
[x.text for x in sample.find("span", class_="location").find_all("a")]

['Brooklyn', 'Washington', 'Houston']

**Get description**

In [24]:
sample.find("p", class_="description").p.text

'Flatiron School offers immersive on-campus and online programs in software engineering, data science, and cybersecurity in NYC, Brooklyn, Washington DC, Houston, Austin, Seattle, Chicago, Denver, and Online. Flatiron School’s immersive courses aim to launch students into fulfilling careers as software engineers, and data scientists through rigorous, market-aligned curricula, and the support of seasoned instructors and personal career coaches. Through test-driven labs and portfolio projects, Flatiron School teaches students to think and build like software engineers and data scientists.'

# 3.1. Create functions to extract individual data points

In [25]:
sample.div.h3.a.text

'1. Flatiron School'

In [26]:
def get_rank(list_item):
    rank_pattern = r"(^\d{1,2})"
    school = list_item.div.h3.a.text
    return int(re.findall(rank_pattern, school)[0])

In [27]:
get_rank(sample)

1

In [28]:
def get_name(list_item):
    name_pattern = r"^\d{1,2}\.\s(.+)"
    school = list_item.div.h3.a.text
    return re.findall(name_pattern, school)[0]

In [29]:
get_name(sample)

'Flatiron School'

In [30]:
sample.find("p", class_="rating-number").text

'Avg Rating: 4.69 (445 reviews)'

In [31]:
def get_rating(list_item):
    rating_pattern = r"^Avg Rating:\s(\d\.\d{1,2})"
    text = list_item.find("p", class_="rating-number").text
    return float(re.findall(rating_pattern, text)[0])

In [32]:
get_rating(sample)

4.69

In [33]:
def get_reviews(list_item):
    review_pattern = r"\((\d+)\sreviews\)$"
    text = list_item.find("p", class_="rating-number").text
    return int(re.findall(review_pattern, text)[0])

In [34]:
get_reviews(sample)

445

In [35]:
sample.find("p", class_="ratings").find_all("span")

[<span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-full_star"></span>,
 <span class="icon-half_star"></span>]

In [36]:
sample.find("p", class_="ratings").find_all("span")[0]

<span class="icon-full_star"></span>

In [37]:
sample.find("p", class_="ratings").find_all("span")[0]["class"]

['icon-full_star']

In [38]:
sample.find("p", class_="ratings").find_all("span")[0]["class"][0]

'icon-full_star'

In [39]:
stars = {"icon-full_star": 1,
         "icon-half_star": .5}

In [40]:
print(stars["icon-full_star"])
print(stars["icon-half_star"])

1
0.5


In [41]:
def get_stars(list_item):
    stars = {"icon-full_star": 1,
             "icon-half_star": .5,
             "icon-empty_star": 0}
    stars_list = list_item.find("p", class_="ratings").find_all("span")
    return sum([stars[star["class"][0]] for star in stars_list])

In [42]:
get_stars(sample)

4.5

In [43]:
[x.text for x in sample.find("span", class_="location").find_all("a")]

['Brooklyn', 'Washington', 'Houston']

In [44]:
def get_locations(list_item):
    return [x.text for x in (list_item
                             .find("span", class_="location")
                             .find_all("a"))]

In [45]:
get_locations(sample)

['Brooklyn', 'Washington', 'Houston']

In [46]:
locs = "|".join(get_locations(sample))

In [47]:
locs.split("|")

['Brooklyn', 'Washington', 'Houston']

In [48]:
def get_locations(list_item):
    return "|".join([x.text for x in (list_item
                                      .find("span", class_="location")
                                      .find_all("a"))])

In [49]:
get_locations(sample)

'Brooklyn|Washington|Houston'

In [50]:
def get_description(list_item):
    return list_item.find("p", class_="description").p.text

In [51]:
get_description(sample)

'Flatiron School offers immersive on-campus and online programs in software engineering, data science, and cybersecurity in NYC, Brooklyn, Washington DC, Houston, Austin, Seattle, Chicago, Denver, and Online. Flatiron School’s immersive courses aim to launch students into fulfilling careers as software engineers, and data scientists through rigorous, market-aligned curricula, and the support of seasoned instructors and personal career coaches. Through test-driven labs and portfolio projects, Flatiron School teaches students to think and build like software engineers and data scientists.'

## 3.2. Create a function that processes an entire list item

In [77]:
import datetime

In [98]:
datetime.datetime.today().strftime("%Y-%m-%d")

'2020-06-15'

In [100]:
def create_row(list_item):
    return {"date_id": datetime.datetime.today().strftime("%Y-%m-%d"),
            "rank": get_rank(list_item),
            "name": get_name(list_item),
            "rating": get_rating(list_item),
            "stars": get_stars(list_item),
            "reviews": get_reviews(list_item),
            "locations": get_locations(list_item),
            "description": get_description(list_item)}

In [101]:
create_row(sample)

{'date_id': '2020-06-15',
 'rank': 1,
 'name': 'Flatiron School',
 'rating': 4.69,
 'stars': 4.5,
 'reviews': 445,
 'locations': 'Brooklyn|Washington|Houston',
 'description': 'Flatiron School offers immersive on-campus and online programs in software engineering, data science, and cybersecurity in NYC, Brooklyn, Washington DC, Houston, Austin, Seattle, Chicago, Denver, and Online. Flatiron School’s immersive courses aim to launch students into fulfilling careers as software engineers, and data scientists through rigorous, market-aligned curricula, and the support of seasoned instructors and personal career coaches. Through test-driven labs and portfolio projects, Flatiron School teaches students to think and build like software engineers and data scientists.'}

In [102]:
[create_row(school) for school in school_list_items]

[{'date_id': '2020-06-15',
  'rank': 1,
  'name': 'Flatiron School',
  'rating': 4.69,
  'stars': 4.5,
  'reviews': 445,
  'locations': 'Brooklyn|Washington|Houston',
  'description': 'Flatiron School offers immersive on-campus and online programs in software engineering, data science, and cybersecurity in NYC, Brooklyn, Washington DC, Houston, Austin, Seattle, Chicago, Denver, and Online. Flatiron School’s immersive courses aim to launch students into fulfilling careers as software engineers, and data scientists through rigorous, market-aligned curricula, and the support of seasoned instructors and personal career coaches. Through test-driven labs and portfolio projects, Flatiron School teaches students to think and build like software engineers and data scientists.'},
 {'date_id': '2020-06-15',
  'rank': 2,
  'name': 'Hack Reactor',
  'rating': 4.7,
  'stars': 4.5,
  'reviews': 279,
  'locations': 'San Francisco|Online|Austin',
  'description': 'Founded in 2012, Hack Reactor is a 12-

In [103]:
df = pd.DataFrame([create_row(school) for school in school_list_items])

In [104]:
df.head()

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,Flatiron School,4.69,4.5,445,Brooklyn|Washington|Houston,Flatiron School offers immersive on-campus and...
1,2020-06-15,2,Hack Reactor,4.70,4.5,279,San Francisco|Online|Austin,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,2020-06-15,3,Codesmith,4.90,5.0,370,Online|Los Angeles|New York City,"Codesmith offers a full-time, 12-week full sta..."
3,2020-06-15,4,App Academy,4.68,4.5,851,San Francisco|Online|New York City,App Academy offers immersive web development c...
4,2020-06-15,5,Turing,4.77,4.5,159,Denver,Turing School of Software & Design is a 7-mont...


In [105]:
def get_rankings(bootcamp):
    url = f"https://www.coursereport.com/best-{bootcamp}-bootcamps"
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content, "html.parser")
    school_list_items = soup.find("ul", id="schools").find_all("li")
    df = pd.DataFrame([create_row(school) for school in school_list_items])
    return df

In [106]:
bootcamps = ["coding", "data-science", "online"]

In [107]:
get_rankings("data-science").head()

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,BrainStation,4.67,4.5,362,Vancouver|Toronto|Online,BrainStation offers full-time and part-time co...
1,2020-06-15,2,Coding Temple,4.92,5.0,124,Online|Washington|Boston,"Coding Temple offers 10-week, full-stack codin..."
2,2020-06-15,3,Divergence Academy,5.00,5.0,20,Dallas,Divergence Academy is a 12-week full-time data...
3,2020-06-15,4,Flatiron School,4.69,4.5,445,Brooklyn|Online|Washington,Flatiron School offers immersive on-campus and...
4,2020-06-15,5,Galvanize,4.47,4.5,177,Los Angeles|Phoenix|Denver,Galvanize offers a 13-week full-time and a 26-...


In [108]:
coding, data_science, online = [get_rankings(ranking) for ranking in bootcamps]

In [109]:
coding.head()

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,Flatiron School,4.69,4.5,445,Brooklyn|Washington|Houston,Flatiron School offers immersive on-campus and...
1,2020-06-15,2,Hack Reactor,4.70,4.5,279,San Francisco|Online|Austin,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,2020-06-15,3,Codesmith,4.90,5.0,370,Online|Los Angeles|New York City,"Codesmith offers a full-time, 12-week full sta..."
3,2020-06-15,4,App Academy,4.68,4.5,851,San Francisco|Online|New York City,App Academy offers immersive web development c...
4,2020-06-15,5,Turing,4.77,4.5,159,Denver,Turing School of Software & Design is a 7-mont...


In [110]:
data_science.head()

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,BrainStation,4.67,4.5,362,Vancouver|Toronto|Online,BrainStation offers full-time and part-time co...
1,2020-06-15,2,Coding Temple,4.92,5.0,124,Online|Washington|Boston,"Coding Temple offers 10-week, full-stack codin..."
2,2020-06-15,3,Divergence Academy,5.00,5.0,20,Dallas,Divergence Academy is a 12-week full-time data...
3,2020-06-15,4,Flatiron School,4.69,4.5,445,Brooklyn|Online|Washington,Flatiron School offers immersive on-campus and...
4,2020-06-15,5,Galvanize,4.47,4.5,177,Los Angeles|Phoenix|Denver,Galvanize offers a 13-week full-time and a 26-...


In [111]:
online.head()

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,AcadGild,4.30,4.0,172,Bangalore|Online,AcadGild is an online coding bootcamp offering...
1,2020-06-15,2,Actualize,4.89,5.0,250,Online|Chicago,Actualize is a 12-week software development bo...
2,2020-06-15,3,Altcademy,4.98,5.0,49,Hong Kong|Online,Altcademy (formerly Hack Pacific) is an educat...
3,2020-06-15,4,App Academy,4.68,4.5,851,Online|San Francisco|New York City,App Academy offers immersive web development c...
4,2020-06-15,5,Barcelona Code School,4.60,4.5,46,Barcelona|Online,Barcelona Code School offers a 9-week full-tim...


# 4. Write rows into MySQL

In [112]:
import pymysql
import getpass

In [130]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       database="bootcamp_rankings",
                       passwd=getpass.getpass())

········


## 4.1. Insert example

In [117]:
example_insert = """
INSERT INTO coding VALUES
    ('2020-06-15',
     1,
     'Flatiron School',
     4.69,
     4.5,
     445,
     'Brooklyn|Washington|Houston',
     'I am a description'),
    ('2020-06-15',
     1,
     'Flatiron School',
     4.69,
     4.5,
     445,
     'Brooklyn|Washington|Houston',
     'I am a description'),
    ('2020-06-15',
     1,
     'Flatiron School',
     4.69,
     4.5,
     445,
     'Brooklyn|Washington|Houston',
     'I am a description');
"""

In [131]:
cursor = conn.cursor()

In [132]:
cursor.execute(example_insert)

3

In [134]:
conn.commit()

In [136]:
conn.close()

## 4.2. Real inserts

write the INSERT statement

In [137]:
coding.head(3)

,date_id,rank,name,rating,stars,reviews,locations,description
0,2020-06-15,1,Flatiron School,4.69,4.5,445,Brooklyn|Washington|Houston,Flatiron School offers immersive on-campus and...
1,2020-06-15,2,Hack Reactor,4.70,4.5,279,San Francisco|Online|Austin,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,2020-06-15,3,Codesmith,4.90,5.0,370,Online|Los Angeles|New York City,"Codesmith offers a full-time, 12-week full sta..."


In [138]:
insert_query = "INSERT INTO coding VALUES "

In [139]:
insert_query

'INSERT INTO coding VALUES '

In [142]:
insert_query + "('2020-06-15',1,'Flatiron School',4.69,4.5,445,'Brooklyn|Washington|Houston','I am a description')," \
             + "('2020-06-15',1,'Flatiron School',4.69,4.5,445,'Brooklyn|Washington|Houston','I am a description')"

"INSERT INTO coding VALUES ('2020-06-15',1,'Flatiron School',4.69,4.5,445,'Brooklyn|Washington|Houston','I am a description'),('2020-06-15',1,'Flatiron School',4.69,4.5,445,'Brooklyn|Washington|Houston','I am a description')"

In [148]:
for idx, row in coding.head(3).iterrows():
    print("-------")
    print(row["name"])

-------
Flatiron School
-------
Hack Reactor
-------
Codesmith


In [165]:
insert_query = "INSERT INTO coding VALUES "

In [166]:
for idx, row in coding.iterrows():
    if idx != 0:
        insert_query = insert_query + ", "
    insert_query = insert_query + str((row["date_id"],
                                       row["rank"],
                                       row["name"],
                                       row["rating"],
                                       row["stars"],
                                       row["reviews"],
                                       row["locations"],
                                       row["description"]))

In [168]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       database="bootcamp_rankings",
                       passwd=getpass.getpass())

········


In [169]:
cursor = conn.cursor()

In [170]:
cursor.execute(insert_query)

48

In [171]:
conn.commit()

In [173]:
conn.close()

## 4.3. Create a insert_rows function

In [179]:
def insert_rows(df, table, conn):
    insert_query = f"INSERT INTO {table} VALUES "
    for idx, row in df.iterrows():
        if idx != 0:
            insert_query = insert_query + ", "
        insert_query = insert_query + str((row["date_id"],
                                           row["rank"],
                                           row["name"],
                                           row["rating"],
                                           row["stars"],
                                           row["reviews"],
                                           row["locations"],
                                           row["description"]))

    cursor = conn.cursor()
    rows = cursor.execute(insert_query)
    conn.commit()
    print(f"{rows} rows written into {table}")

In [175]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       database="bootcamp_rankings",
                       passwd=getpass.getpass())

········


In [176]:
insert_rows(coding, "coding", conn)

48 written into coding


In [180]:
insert_rows(data_science, "data_science", conn)

15 rows written into data_science


In [181]:
insert_rows(online, "online", conn)

35 rows written into online


In [182]:
import example

example


In [183]:
bootcamps

['coding', 'data-science', 'online']

In [184]:
{ranking: get_rankings(ranking) for ranking in bootcamps}

{'coding':        date_id  rank                        name  rating  stars  reviews  \
 0   2020-06-15     1             Flatiron School    4.69    4.5      445   
 1   2020-06-15     2                Hack Reactor    4.70    4.5      279   
 2   2020-06-15     3                   Codesmith    4.90    5.0      370   
 3   2020-06-15     4                 App Academy    4.68    4.5      851   
 4   2020-06-15     5                      Turing    4.77    4.5      159   
 5   2020-06-15     6           Fullstack Academy    4.89    5.0      272   
 6   2020-06-15     7            General Assembly    4.31    4.5      455   
 7   2020-06-15     8               Tech Elevator    4.96    5.0      226   
 8   2020-06-15     9               DigitalCrafts    4.88    5.0      213   
 9   2020-06-15    10              Software Guild    4.67    4.5      147   
 10  2020-06-15    11            Holberton School    4.45    4.5       78   
 11  2020-06-15    12                     Wyncode    4.74    4.5  